In [13]:
import os
import cv2
import matplotlib
import matplotlib.pyplot as plt
import h5py 

from tqdm import tqdm

from holobot.utils.files import *
from holobot.constants import *

In [14]:


def plot_line(X1, X2, Y1, Y2):
    plt.plot([X1, X2], [Y1, Y2])

class PlotHand2D():
    def __init__(self):

        # Thumb bound info
        self.thumb_bounds = None
        self.thumb_bounds_path = VR_DISPLAY_THUMB_BOUNDS_PATH
        self.bound_update_counter = 0
        self._check_thumb_bounds()

        # Checking image storage path
        # make_dir(os.path.join(PLOT_PATH))
        matplotlib.use('Agg')

        # Figure settings
        self.fig = plt.figure(figsize=(6, 6), dpi=60)

    def _check_thumb_bounds(self):
        if check_file(self.thumb_bounds_path):
            self.thumb_bounds = get_npz_data(self.thumb_bounds_path)

    def _set_limits(self):
        plt.axis([-0.12, 0.12, -0.02, 0.2])

    def _draw_thumb_bounds(self):
        for idx in range(VR_THUMB_BOUND_VERTICES):
            plot_line(
                self.thumb_bounds[idx][0], 
                self.thumb_bounds[(idx + 1) % VR_THUMB_BOUND_VERTICES][0], 
                self.thumb_bounds[idx][1], 
                self.thumb_bounds[(idx + 1) % VR_THUMB_BOUND_VERTICES][1]
            )
        
    def draw_hand(self, X, Y):
        plt.plot(X, Y, 'ro')

        if self.thumb_bounds is not None:
            self._draw_thumb_bounds()

        # Drawing connections fromn the wrist - 0
        for idx in OCULUS_JOINTS['metacarpals']:
            plot_line(X[0], X[idx], Y[0], Y[idx])

        # Drawing knuckle to knuckle connections and knuckle to finger connections
        for key in ['knuckles', 'thumb', 'index', 'middle', 'ring', 'pinky']:
            for idx in range(len(OCULUS_JOINTS[key]) - 1):
                plot_line(
                    X[OCULUS_JOINTS[key][idx]], 
                    X[OCULUS_JOINTS[key][idx + 1]], 
                    Y[OCULUS_JOINTS[key][idx]], 
                    Y[OCULUS_JOINTS[key][idx + 1]]
                )

    def draw(self, X, Y, fig_path=None):
        # Setting the plot limits
        self._set_limits()

        # Resetting the thumb bounds
        if self.bound_update_counter % 10 == 0:
            self._check_thumb_bounds()
            self.bound_update_counter = 0
        else:
            self.bound_update_counter += 1

        # Plotting the lines to visualize the hand
        self.draw_hand(X, Y)

        # Saving and obtaining the plot
        plt.savefig(fig_path)

        # Resetting and pausing the 3D plot
        # plt.pause(0.001) # This make the graph show up if matplotlib is in Tkinter mode
        plt.cla()

In [15]:

def dump_states(root, keypoint_indices, image_indices, view_num=0):
    # Make directory to dump the visualization
    pbar = tqdm(total=len(keypoint_indices))

    with h5py.File(os.path.join(root, 'keypoints.h5'), 'r') as f:
        hand_keypoints = f['transformed_hand_coords'][()]

    viz_dir = os.path.join(root, 'visualization')
    os.makedirs(viz_dir, exist_ok=True)

    # Get the hand plotter
    hand_plotter = PlotHand2D()
    
    video_path = os.path.join(root, f'cam_{view_num}_rgb_video.avi')
    vidcap = cv2.VideoCapture(video_path)
    success, image = vidcap.read()
    frame_count = int(vidcap.get(cv2.CAP_PROP_FRAME_COUNT))
    frame_id = 0
    for i in range(len(keypoint_indices)):
        [_, keypoint_id], [_, image_id] = keypoint_indices[i], image_indices[i]
        while frame_id != image_id and success:
            # Find the frame that is equal to image_id
            success, image = vidcap.read()
            frame_id += 1
        dump_demo_state(
            frame_id = i,
            viz_dir = viz_dir,
            hand_plotter = hand_plotter,
            keypoint_values = hand_keypoints[keypoint_id,:],
            camera_img = image
        )

        pbar.update(1)

    pbar.close()

def dump_demo_state(frame_id, viz_dir, hand_plotter, keypoint_values, camera_img):
    hand_plotter.draw(
        keypoint_values[:, 0], keypoint_values[:, 1],
        fig_path=os.path.join(viz_dir, 'hand_coords.png'))

    hand_coord_img = cv2.imread(os.path.join(viz_dir, 'hand_coords.png'))
    height_scale = camera_img.shape[0] / hand_coord_img.shape[0]
    hand_coord_img = cv2.resize(
        hand_coord_img,
        (int(hand_coord_img.shape[1] * height_scale),
         int(hand_coord_img.shape[0] * height_scale))
    )
    total_img = cv2.hconcat([camera_img, hand_coord_img])

    img_name = 'state_{}.png'.format(str(frame_id).zfill(3))
    cv2.imwrite(os.path.join(viz_dir, img_name), total_img)



In [16]:
def load_indices(root):
    with open(os.path.join(root, 'keypoint_indices.pkl'), 'rb') as f:
        keypoint_indices = pickle.load(f)

    with open(os.path.join(root, 'image_indices.pkl'), 'rb') as f:
        image_indices = pickle.load(f)

    return keypoint_indices, image_indices

In [17]:
import glob
data_path = '/home/irmak/Workspace/Holo-Bot/extracted_data/human_sponge_flipping'
roots = glob.glob(f'{data_path}/demonstration_*')

view_num = 0
for root in roots:
    keypoint_indices, image_indices = load_indices(root)
    print('keypoint_indices: {}, image_indices: {}'.format(keypoint_indices, image_indices))
    dump_states(root, keypoint_indices, image_indices, view_num=view_num)


keypoint_indices: [[0, 0], [0, 13], [0, 25], [0, 37], [0, 49], [0, 61], [0, 73], [0, 85], [0, 97], [0, 109], [0, 121], [0, 133], [0, 145], [0, 157], [0, 169], [0, 181], [0, 192], [0, 204], [0, 216], [0, 228], [0, 240], [0, 252], [0, 264], [0, 276], [0, 288], [0, 300], [0, 312], [0, 324], [0, 336], [0, 348], [0, 360], [0, 372], [0, 384], [0, 396], [0, 408], [0, 420], [0, 432], [0, 444], [0, 456], [0, 468], [0, 480], [0, 492], [0, 504], [0, 516], [0, 528], [0, 540], [0, 552], [0, 564], [0, 576], [0, 588], [0, 599], [0, 610], [0, 623], [0, 635], [0, 647], [0, 659], [0, 671], [0, 683], [0, 695], [0, 707], [0, 719], [0, 731], [0, 743], [0, 755], [0, 767], [0, 779], [0, 791], [0, 803], [0, 815], [0, 827], [0, 839], [0, 851], [0, 863], [0, 875], [0, 887], [0, 898]], image_indices: [[0, 530], [0, 536], [0, 542], [0, 548], [0, 554], [0, 560], [0, 566], [0, 572], [0, 578], [0, 584], [0, 590], [0, 596], [0, 602], [0, 608], [0, 614], [0, 620], [0, 626], [0, 632], [0, 638], [0, 644], [0, 650], [0, 

100%|██████████| 76/76 [00:10<00:00,  7.00it/s]


keypoint_indices: [[1, 0], [1, 12], [1, 24], [1, 36], [1, 48], [1, 60], [1, 72], [1, 84], [1, 96], [1, 108], [1, 120], [1, 132], [1, 144], [1, 156], [1, 168], [1, 180], [1, 192], [1, 204], [1, 216], [1, 228], [1, 240], [1, 252], [1, 264], [1, 276], [1, 288], [1, 300], [1, 312], [1, 324], [1, 336], [1, 348], [1, 360], [1, 372], [1, 383], [1, 395], [1, 407], [1, 419], [1, 431], [1, 443], [1, 455], [1, 467], [1, 479], [1, 491], [1, 503], [1, 515], [1, 527], [1, 539], [1, 551], [1, 563], [1, 575], [1, 587], [1, 599], [1, 611], [1, 623], [1, 635], [1, 647], [1, 659], [1, 671], [1, 683], [1, 695], [1, 707], [1, 719], [1, 731], [1, 743], [1, 755], [1, 767], [1, 779], [1, 791], [1, 803], [1, 813], [1, 825], [1, 837], [1, 849], [1, 861], [1, 873], [1, 885], [1, 897], [1, 909], [1, 921], [1, 933], [1, 945], [1, 957], [1, 969], [1, 981], [1, 993], [1, 999]], image_indices: [[1, 4], [1, 10], [1, 16], [1, 22], [1, 28], [1, 34], [1, 40], [1, 46], [1, 52], [1, 58], [1, 64], [1, 70], [1, 76], [1, 82],

100%|██████████| 85/85 [00:11<00:00,  7.13it/s]


In [18]:
# Turn the images to a video and delete the directory
video_fps = 8
for root in roots:
    print('dumping video in root: {}'.format(root))
    video_path = os.path.join(root, f'visualization_{view_num}.mp4')
    if os.path.exists(video_path):
        os.remove(video_path)
    viz_dir = os.path.join(root, 'visualization')
    os.system('ffmpeg -r {} -i {}/%*.png -vf scale=2000x720,setsar=1:1 {}'.format(
        video_fps, # fps
        viz_dir,
        video_path
    ))


dumping video in root: /home/irmak/Workspace/Holo-Bot/extracted_data/human_sponge_flipping/demonstration_1


ffmpeg version 5.1.2 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 10.4.0 (conda-forge gcc 10.4.0-18)
  configuration: --prefix=/home/irmak/miniconda3/envs/tactile_learning --cc=/home/conda/feedstock_root/build_artifacts/ffmpeg_1666357487580/_build_env/bin/x86_64-conda-linux-gnu-cc --cxx=/home/conda/feedstock_root/build_artifacts/ffmpeg_1666357487580/_build_env/bin/x86_64-conda-linux-gnu-c++ --nm=/home/conda/feedstock_root/build_artifacts/ffmpeg_1666357487580/_build_env/bin/x86_64-conda-linux-gnu-nm --ar=/home/conda/feedstock_root/build_artifacts/ffmpeg_1666357487580/_build_env/bin/x86_64-conda-linux-gnu-ar --disable-doc --disable-openssl --enable-demuxer=dash --enable-hardcoded-tables --enable-libfreetype --enable-libfontconfig --enable-libopenh264 --enable-gnutls --enable-libmp3lame --enable-libvpx --enable-pthreads --enable-vaapi --enable-gpl --enable-libx264 --enable-libx265 --enable-libaom --enable-libsvtav1 --enable-libxml2 --enable-pic --enable-shared --disable-

dumping video in root: /home/irmak/Workspace/Holo-Bot/extracted_data/human_sponge_flipping/demonstration_2


[libx264 @ 0x55c9549d3d00] using SAR=1/1
[libx264 @ 0x55c9549d3d00] using cpu capabilities: MMX2 SSE2Fast SSSE3 SSE4.2 AVX FMA3 BMI2 AVX2
[libx264 @ 0x55c9549d3d00] profile High 4:4:4 Predictive, level 4.0, 4:4:4, 8-bit
[libx264 @ 0x55c9549d3d00] 264 - core 164 r3095 baee400 - H.264/MPEG-4 AVC codec - Copyleft 2003-2022 - http://www.videolan.org/x264.html - options: cabac=1 ref=3 deblock=1:0:0 analyse=0x3:0x113 me=hex subme=7 psy=1 psy_rd=1.00:0.00 mixed_ref=1 me_range=16 chroma_me=1 trellis=1 8x8dct=1 cqm=0 deadzone=21,11 fast_pskip=1 chroma_qp_offset=4 threads=22 lookahead_threads=3 sliced_threads=0 nr=0 decimate=1 interlaced=0 bluray_compat=0 constrained_intra=0 bframes=3 b_pyramid=2 b_adapt=1 b_bias=0 direct=1 weightb=1 open_gop=0 weightp=2 keyint=250 keyint_min=8 scenecut=40 intra_refresh=0 rc_lookahead=40 rc=crf mbtree=1 crf=23.0 qcomp=0.60 qpmin=0 qpmax=69 qpstep=4 ip_ratio=1.40 aq=1:1.00
Output #0, mp4, to '/home/irmak/Workspace/Holo-Bot/extracted_data/human_sponge_flipping/dem

In [19]:
# Remove the img directory
import shutil
for root in roots:
    viz_dir = os.path.join(root, 'visualization')
    shutil.rmtree(viz_dir)
